<a href="https://colab.research.google.com/github/Sebastian310/Modelos-Predictivos/blob/main/Pararelo_muticlase_BINARIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importar librerias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

warnings.filterwarnings('ignore')

#importar datos y observar el tipo de variables

df = pd.read_excel("DataTFM.xlsx").dropna()
#df.info()

# Crear columnas para resultados del local en fechas anteriores (variables rezagadas)

df['Team_Points(T-1)'] = df.groupby('Team')['Team_Points'].shift(1)
df['Team_Points(T-2)'] = df.groupby('Team')['Team_Points'].shift(2)
df['Team_Points(T-3)'] = df.groupby('Team')['Team_Points'].shift(3)
df['Team_Points(T-4)'] = df.groupby('Team')['Team_Points'].shift(4)
df['Team_Points(T-5)'] = df.groupby('Team')['Team_Points'].shift(5)

#Creo nuevo dataframes dfhome y dfaway que contienen solo los datos Away y home y reseteamos el indice para evitar que el indice del datafrema original se agregue a los nuevos

dfaway = df[df["Location"]=="Away"].reset_index(drop=True)
dfhome = df[df["Location"]=="Home"].sort_values(by=["Opponent","season","Round"]).reset_index(drop=True)

# Crear columnas para resultados del oponente en fechas anteriores (variables rezagadas)

dfhome['Opp_Points(T-1)'] = dfaway['Team_Points(T-1)']
dfhome['Opp_Points(T-2)'] = dfaway['Team_Points(T-2)']
dfhome['Opp_Points(T-3)'] = dfaway['Team_Points(T-3)']
dfhome['Opp_Points(T-4)'] = dfaway['Team_Points(T-4)']
dfhome['Opp_Points(T-5)'] = dfaway['Team_Points(T-5)']

#Creando el nuevo set con todas las variables, descartando las que no se usaran y reorganizando.

newdf = dfhome.sort_values(by=["Date","Round"]).reset_index(drop=True)
col_eliminar = ["Time","Date", "Location","Opponent_Points","Country","Competition","Team_Score","Opponent_Score"]
newdf.drop(col_eliminar, axis=1,inplace=True)
newnames = {"Team_Points": "Result", "team_accum_points":"Team_accum_points","opp_accum_goals":"Opp_accum_goals","opp_accum_points":"Opp_accum_points","season":"season"}
newdf.rename(columns=newnames, inplace=True)
#newdf.info()

# Binarizo el resultado de la recien renombrada variable independiente "Result"
# 0 y 1 = 1 perdió
# 3= 0 Gan{o}

newdf['Result'] = newdf['Result'].replace({0: 1, 1: 1,3:0})


# Reorganizo el Datafreme Home

nuevo_orden_columnas = ['Round', 'Team', 'Team_accum_goals', 'Team_accum_points', 'Opponent',
                         'Opp_accum_goals', 'Opp_accum_points',
                        'season', 'Team_Points(T-1)', 'Team_Points(T-2)', 'Team_Points(T-3)',
                        'Team_Points(T-4)', 'Team_Points(T-5)', 'Opp_Points(T-1)',
                        'Opp_Points(T-2)', 'Opp_Points(T-3)', 'Opp_Points(T-4)',
                        'Opp_Points(T-5)', 'Result']

newdf = newdf.reindex(columns=nuevo_orden_columnas)

#Elimino Valores Vac{ios en Team_Points(T-5) y Opp_Points(T-5)

newdf = newdf.dropna(subset=['Team_Points(T-5)', 'Opp_Points(T-5)'])



# ■■■ Genero excel para validar adicional ( esto no es necesario)
#newdf.to_excel('nuevo_orden_columnas.xlsx', index=False)
newdf.info()

# Crear variables dummies para 'Team' y 'Opponent'

team_dummies = pd.get_dummies(newdf['Team'], prefix='Team')
opponent_dummies = pd.get_dummies(newdf['Opponent'], prefix='Opponent')

# Concatenar las variables dummies al dataframe original
X = pd.concat([newdf, team_dummies, opponent_dummies], axis=1)

# Eliminar las columnas originales 'Team' y 'Opponent'
X = newdf.drop(['Team', 'Opponent'], axis=1)


# Defino variables dependientes e independientes

# Aquí estás creando X, que es un DataFrame que contiene tus variables independientes. Estas son las variables que usarás para predecir y. El método drop('Result', axis=1) está eliminando la columna ‘Result’ de tu DataFrame newdf, dejándote con un DataFrame que contiene solo las variables independientes.
X = newdf.drop('Result', axis=1)


y = newdf["Result"]

#Para ver el conjunto de las X
X.head()

# PAra ver las Y

y.head()


# Divido los datos en conjuntos de entrenamiento y prueba 80%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# OPCION 1A Regresion

# Creo el modelo de regresión logística multinomial
model = LogisticRegression()

# Entren0 el modelo
model.fit(X_train, y_train)

# Realiza predicciones en conjunto de prueba
y_pred = model.predict(X_test)

# Se calcula la precisión del modelo comparando las predicciones con los valores reales.
accuracy = accuracy_score(y_test, y_pred)

print(" Presici{on del modelo :", accuracy)

# sELECCIONO MANUALMENTO SUBCONJUNTOS DE DATOS REGRESION LOGISTICA

# Selecciono las primeras 1000 filas para el conjunto de entrenamiento
X_train = X.iloc[:1000]
y_train = y.iloc[:1000]

# El resto de los datos se usan para el conjunto de prueba
X_test = X.iloc[1000:2000]
y_test = y.iloc[1000:2000]

# OPCION 1A Regresion

# Creo el modelo de regresión logística multinomial
model = LogisticRegression()

# Entreno el modelo
model.fit(X_train, y_train)

# Realiza predicciones en conjunto de prueba
y_pred = model.predict(X_test)

# Se calcula la precisión del modelo comparando las predicciones con los valores reales.
accuracy = accuracy_score(y_test, y_pred)

print("Presición del modelo :", accuracy)




